# Score Function to optimize the path of UUV
provide score fields for different cases ('consideration') for UUV sim

In [ ]:
import xarray as xr
import numpy as np

## Score function to provide score for each case
INPUT: [lon, lat, depth, timestep]
OUTPUT: [score1, score2, score3]

1. gradient of O2 satuation rate: larger magnitude of gradient has higher score, need O2, water age, temperature, and salinity. gradient_O2 = abs(O2sat(Potential Coords) - O2sat(Current Coords)).
2. Current: ?

In [ ]:
def get_o2_age(lon, lat, depth, method='linear'):
    " get Oxygen [mmol/(m^3 s)] and water age from OCIM at a giving location"

    OCIM_nc_file = 'ds_OCIM_struct.nc' # in 'data' folder, update the nc file if needed
    ds = xr.open_dataset(OCIM_nc_file)

    point = ds.sel(longitude=lon, latitude=lat, depth=depth, method=method)
    
    O2 = float(point['O2'].values)
    age = float(point['age'].values)
    
    return O2, age

In [ ]:
def get_o2sat(lon, lat, depth, method='linear'):
    """
    Computes the oxygen saturation concentration at 1 atm total pressure
    in mol/m^3 given temperature (T, in °C) and salinity (S, in ‰).

    From Garcia and Gordon (1992), Limnology and Oceanography.
    Note: The "A3*TS^2" term in the paper is incorrect.

    Valid range:
        T(freezing) <= T <= 40 °C
        0 ‰ <= S <= 42 ‰

    Check value:
        T = 10.0 °C, S = 35.0 ‰ → o2sat = 0.282015 mol/m^3 = 282 mmol/m^3
    """

    # --------------------------------------------------------------------------
    # grab T and S from location (lon, lat, depth) 
    # --------------------------------------------------------------------------

    HYCOM_nc_file = 'copernicus-subset-jul-01-03-2025.nc' # in 'data' folder, update the nc file if needed
    ds = xr.open_dataset(HYCOM_nc_file)

    point = ds.sel(longitude=lon, latitude=lat, depth=depth, method=method)

    T = float(point['temperature'].values) # unit of deg c
    S = float(point['salinity'].values) # unit of PSU
    

    # -------------------------------------------------------------------------
    # calculate the satuation rate
    # --------------------------------------------------------------------------
    
    # Coefficients
    A0, A1, A2, A3, A4, A5 = 2.00907, 3.22014, 4.05010, 4.94457, -2.56847e-1, 3.88767
    B0, B1, B2, B3 = -6.24523e-3, -7.37614e-3, -1.03410e-2, -8.17083e-3
    C0 = -4.88682e-7

    TT = 298.15 - T
    TK = 273.15 + T
    TS = np.log(TT / TK)

    # Polynomial terms
    TS2, TS3, TS4, TS5 = TS**2, TS**3, TS**4, TS**5

    # Oxygen solubility (ml/L)
    CO = (A0 + A1*TS + A2*TS2 + A3*TS3 + A4*TS4 + A5*TS5
          + S*(B0 + B1*TS + B2*TS2 + B3*TS3)
          + C0*(S**2))

    o2sat = np.exp(CO)

    # Convert from ml/L to mmol/m^3
    o2sat = (o2sat / 22391.6) * 1000.0 * 1000.0 

    return o2sat

In [ ]:
# Example usage
robot_location = (123.45, 37.78, 10.0)  # (lon, lat, depth)
O2, age = get_o2_age(*robot_location)
O2sat = get_o2sat(*robot_location)

print("O2:", O2)
print("Age:", age)
print("O2_Sat:", O2sat)

In [1]:
def score_o2sat(cc, pc):
    """
    input:
    ---------------------------
    cc: tuple
        current coords (lon0, lat0, depth0)
    pc: list of tuples
        potential coords [(lon1, lat1, depth1), (lon2,lat2, depth2), ....]

    get O2sat_{} based on the lon, lat, depth.
    
    gradient = np.abs(O2sat_cc - O2sat_pc)

    normalized to 0-1 as score

    return:
    ---------------------------
    normalized gradient scores (0-1), with the size of pc
    
    """
    # AOR in current location
    O2sat_cc = get_o2sat(*cc)

    # AOR in potential locations
    O2sat_pc = np.array([get_o2sat(*loc) for loc in pc])

    grad = np.abs(O2sat_cc - O2sat_pc)

    # normalize to 0-1
    total = np.sum(grad)
    scores = grad / total if total > 0 else np.zeros_like(grad)

    return scores

In [ ]:
# example usage

cc = (lon0, lat0, depth0)
pc = [(lon1, lat1, depth1), (lon2, lat2, depth2), (lon3, lat3, depth3)]

scores = score_o2sat(cc, pc)
print(scores)  # array of normalized scores